### 1. Imports

In [1]:
import os
import pandas as pd
import numpy as np
from lcfunctions import download_alerce_lc, download_lasair_lc, display_lightcurve
from lcfunctions import lasair_clean, load_lasair_lc
from scipy.signal import find_peaks
from scipy.stats import linregress
from featureextractor import FeatureExtractor
import warnings
warnings.filterwarnings("ignore")

### 2. Load dataframe of AAVSO CVs in ZTF

In [2]:
# Load dataset
df_ZTF_CVs = pd.read_csv('../processed_data/AAVSOCVsZTFxmatch_21032023_BTS_labelled.csv')
# test = pd.read_csv('../processed_data/AAVSOCVsZTFxmatch_21032023_BTS_manual_labelled.xlsm')

df_ZTF_CVs.CV_Types.value_counts()

dwarf_nova            4822
nova_like              263
polar                  126
nova                    88
intermediate_polar      54
AMCVn                   46
Name: CV_Types, dtype: int64

In [3]:
# List of objects.
lst = df_ZTF_CVs.Xmatch_obj.to_list()
# Check length of objects list
print(f'Length of list of objects: {len(lst)}')

Length of list of objects: 5708


### Display light curves

In [4]:
date = '2023_03_25'

manuallist = False

# Apply errorbars
error = False

if error == True:
        errCollasair1 = 'sigmapsf'
        errCollasair2 = 'dc_sigmag'
else:
    errCollasair1 = None
    errCollasair2 = None

objlist = df_ZTF_CVs[df_ZTF_CVs['Type'].str.contains('UGSS', na=False)]['Xmatch_obj'].iloc[20:40]
objtype = df_ZTF_CVs[df_ZTF_CVs['Type'].str.contains('UGSS', na=False)]['Type'].iloc[20:40]

if manuallist==True:
     objlist = ['ZTF18abajshu','ZTF18aagrgoe','ZTF18abchhxd','ZTF18aabfcyi','ZTF18abnzbxh','ZTF18acrsmyu','ZTF21abhrevv',
                'ZTF18abavrji','ZTF18aanwrti','ZTF18aammncw','ZTF19abzzuin','ZTF18acyerom','ZTF20abgbxco','ZTF18achemir',
                'ZTF18abmaqoi','ZTF18adlkmio','ZTF18abiwtyk','ZTF18aaueblh','ZTF18aakvuwy','ZTF18abcnbki','ZTF18abakkox']

i=0
for object, type in zip(objlist, objtype):
    print(i, object, type)
    
    # Display lasair difference magnitude and apparent magnitude light curves.
    lc_lasair = load_lasair_lc(object, path=f'../lightcurves_dataset/lasair_{date}')
    lc_lasair_appmag = lasair_clean(lc_lasair, limit=25, magerrlim=1)

    # print('Lasair difference magnitude')
    # display_lightcurve(lc_lasair,x='jd',y='magpsf',errorCol=errCollasair1)
    print('Lasair apparent magnitude')
    # display_lightcurve(lc_lasair_appmag,x='jd',y='dc_mag',errorCol=errCollasair2)
    i+=1


0 ZTF18abakkox UGSS
Lasair apparent magnitude
1 ZTF18aabfalx UGSS
Lasair apparent magnitude
2 ZTF18abcplfr UGSS
Lasair apparent magnitude
3 ZTF17aabpkbr UGSS
Lasair apparent magnitude
4 ZTF18abmwrrx UGSS
Lasair apparent magnitude
5 ZTF18aazwdkw UGSS
Lasair apparent magnitude
6 ZTF18abaeinc UGSS
Lasair apparent magnitude
7 ZTF18abcktct UGSS
Lasair apparent magnitude
8 ZTF18abckswk UGSS
Lasair apparent magnitude
9 ZTF18abcpkyg UGSS
Lasair apparent magnitude
10 ZTF18abflqfh UGSS
Lasair apparent magnitude
11 ZTF18abmmwiy UGSS
Lasair apparent magnitude
12 ZTF18adkpgdq UGSS
Lasair apparent magnitude
13 ZTF18aayhtdm UGSS
Lasair apparent magnitude
14 ZTF18aazvhdb UGSS
Lasair apparent magnitude
15 ZTF18abcnbki UGSS
Lasair apparent magnitude
16 ZTF18abgyoaa UGSS
Lasair apparent magnitude
17 ZTF18abcoxou UGSS
Lasair apparent magnitude
18 ZTF18aberfsf UGSS
Lasair apparent magnitude
19 ZTF18aasncio UGSS
Lasair apparent magnitude


### Formulate functions for the features you wish to extract.

### Test some features

In [10]:
from scipy.signal import find_peaks
from scipy.stats import linregress
import numpy as np

# objlist = ['ZTF18aanwrti','ZTF18abmaqoi','ZTF18aabfcyi','ZTF18aakvuwy','ZTF18abajshu','ZTF20abgbxco','ZTF21abhrevv',
#            'ZTF18aagrgoe','ZTF18abchhxd']
feature_df = pd.DataFrame()
for obj in objlist:
    type = df_ZTF_CVs[df_ZTF_CVs['Xmatch_obj']==obj]['Type'].iloc[0]
    print(obj, type)

    # Load and process lasair light curve
    lc_test = load_lasair_lc(oid=obj, path=f'../lightcurves_dataset/lasair_{date}')
    lc_appmag_test = lasair_clean(lc_test, limit=25, magerrlim=1)

    # Create a copy of the light curve
    lc = lc_appmag_test.copy()

    fe = FeatureExtractor(lc=lc)
    feets = fe.extract_feets()
    custom = fe.extract_custom()
    
    # Conactenate custom features to feets
    features_single = pd.concat([feets, custom], axis=1)

    # Add features to dataframe
    feature_df = feature_df.append(features_single, ignore_index=True)


  

ZTF18abakkox UGSS
ZTF18aabfalx UGSS
ZTF18abcplfr UGSS
ZTF17aabpkbr UGSS
ZTF18abmwrrx UGSS
ZTF18aazwdkw UGSS
ZTF18abaeinc UGSS
ZTF18abcktct UGSS
ZTF18abckswk UGSS
ZTF18abcpkyg UGSS
ZTF18abflqfh UGSS
ZTF18abmmwiy UGSS
ZTF18adkpgdq UGSS
ZTF18aayhtdm UGSS
ZTF18aazvhdb UGSS
ZTF18abcnbki UGSS
ZTF18abgyoaa UGSS
ZTF18abcoxou UGSS
ZTF18aberfsf UGSS
ZTF18aasncio UGSS


In [11]:
pd.options.display.max_columns=None
feature_df

,Amplitude_g,AndersonDarling_g,Autocor_length_g,Beyond1Std_g,CAR_mean_g,CAR_sigma_g,CAR_tau_g,Con_g,Eta_e_g,FluxPercentileRatioMid20_g,FluxPercentileRatioMid35_g,FluxPercentileRatioMid50_g,FluxPercentileRatioMid65_g,FluxPercentileRatioMid80_g,Freq1_harmonics_amplitude_0_g,Freq1_harmonics_amplitude_1_g,Freq1_harmonics_amplitude_2_g,Freq1_harmonics_amplitude_3_g,Freq1_harmonics_rel_phase_0_g,Freq1_harmonics_rel_phase_1_g,Freq1_harmonics_rel_phase_2_g,Freq1_harmonics_rel_phase_3_g,Freq2_harmonics_amplitude_0_g,Freq2_harmonics_amplitude_1_g,Freq2_harmonics_amplitude_2_g,Freq2_harmonics_amplitude_3_g,Freq2_harmonics_rel_phase_0_g,Freq2_harmonics_rel_phase_1_g,Freq2_harmonics_rel_phase_2_g,Freq2_harmonics_rel_phase_3_g,Freq3_harmonics_amplitude_0_g,Freq3_harmonics_amplitude_1_g,Freq3_harmonics_amplitude_2_g,Freq3_harmonics_amplitude_3_g,Freq3_harmonics_rel_phase_0_g,Freq3_harmonics_rel_phase_1_g,Freq3_harmonics_rel_phase_2_g,Freq3_harmonics_rel_phase_3_g,Gskew_g,LinearTrend_g,MaxSlope_g,Mean_g,Meanvariance_g,MedianAbsDev_g,MedianBRP_g,PairSlopeTrend_g,PercentAmplitude_g,PercentDifferenceFluxPercentile_g,PeriodLS_g,Period_fit_g,Psi_CS_g,Psi_eta_g,Q31_g,Rcs_g,Skew_g,SlottedA_length_g,SmallKurtosis_g,Std_g,StetsonK_g,StetsonK_AC_g,StructureFunction_index_21_g,StructureFunction_index_31_g,StructureFunction_index_32_g,Amplitude_r,AndersonDarling_r,Autocor_length_r,Beyond1Std_r,CAR_mean_r,CAR_sigma_r,CAR_tau_r,Con_r,Eta_e_r,FluxPercentileRatioMid20_r,FluxPercentileRatioMid35_r,FluxPercentileRatioMid50_r,FluxPercentileRatioMid65_r,FluxPercentileRatioMid80_r,Freq1_harmonics_amplitude_0_r,Freq1_harmonics_amplitude_1_r,Freq1_harmonics_amplitude_2_r,Freq1_harmonics_amplitude_3_r,Freq1_harmonics_rel_phase_0_r,Freq1_harmonics_rel_phase_1_r,Freq1_harmonics_rel_phase_2_r,Freq1_harmonics_rel_phase_3_r,Freq2_harmonics_amplitude_0_r,Freq2_harmonics_amplitude_1_r,Freq2_harmonics_amplitude_2_r,Freq2_harmonics_amplitude_3_r,Freq2_harmonics_rel_phase_0_r,Freq2_harmonics_rel_phase_1_r,Freq2_harmonics_rel_phase_2_r,Freq2_harmonics_rel_phase_3_r,Freq3_harmonics_amplitude_0_r,Freq3_harmonics_amplitude_1_r,Freq3_harmonics_amplitude_2_r,Freq3_harmonics_amplitude_3_r,Freq3_harmonics_rel_phase_0_r,Freq3_harmonics_rel_phase_1_r,Freq3_harmonics_rel_phase_2_r,Freq3_harmonics_rel_phase_3_r,Gskew_r,LinearTrend_r,MaxSlope_r,Mean_r,Meanvariance_r,MedianAbsDev_r,MedianBRP_r,PairSlopeTrend_r,PercentAmplitude_r,PercentDifferenceFluxPercentile_r,PeriodLS_r,Period_fit_r,Psi_CS_r,Psi_eta_r,Q31_r,Rcs_r,Skew_r,SlottedA_length_r,SmallKurtosis_r,Std_r,StetsonK_r,StetsonK_AC_r,StructureFunction_index_21_r,StructureFunction_index_31_r,StructureFunction_index_32_r,Eta_color,Q31_color,StetsonJ,StetsonL,median_g,min_mag_g,max_mag_g,n_obs_g,dif_min_mean_g,dif_min_median_g,dif_max_mean_g,dif_max_median_g,dif_max_min_g,temporal_baseline_g,avg_obs_per_day_g,kurtosis_g,pwr_max_g,freq_pwr_max_g,FalseAlarm_prob_g,pwr_maxovermean_g,npeaks_pt5to1_g,rrate_pt5to1_g,drate_pt5to1_g,amp_pt5to1_g,npeaks_1to2_g,rrate_1to2_g,drate_1to2_g,amp_1to2_g,npeaks_2to5_g,rrate_2to5_g,drate_2to5_g,amp_2to5_g,npeaks_above5_g,rrate_above5_g,drate_above5_g,amp_above5_g,rollstd_ratio_t20s10_g,stdstilllev_t20s10g,rollstd_ratio_t10s5_g,stdstilllev_t10s5g,pnts_leq_rollMedWin20-1mag_g,pnts_leq_rollMedWin20-2mag_g,pnts_leq_rollMedWin20-5mag_g,pnts_geq_rollMedWin20+1mag_g,pnts_geq_rollMedWin20+2mag_g,pnts_geq_rollMedWin20+3mag_g,pnts_leq_median-1mag_g,pnts_leq_median-2mag_g,pnts_leq_median-5mag_g,pnts_geq_median+1mag_g,pnts_geq_median+2mag_g,pnts_geq_median+3mag_g,median_r,min_mag_r,max_mag_r,n_obs_r,dif_min_mean_r,dif_min_median_r,dif_max_mean_r,dif_max_median_r,dif_max_min_r,temporal_baseline_r,avg_obs_per_day_r,kurtosis_r,pwr_max_r,freq_pwr_max_r,FalseAlarm_prob_r,pwr_maxovermean_r,npeaks_pt5to1_r,rrate_pt5to1_r,drate_pt5to1_r,amp_pt5to1_r,npeaks_1to2_r,rrate_1to2_r,drate_1to2_r,amp_1to2_r,npeaks_2to5_r,rrate_2to5_r,drate_2to5_r,amp_2to5_r,npeaks_above5_r,rrate_above5_r,drate_above5_r,amp_above5_r,rollstd_ratio_t20s1

In [8]:
# plot the lomb-scargle periodogram of the light curve
from astropy.timeseries import LombScargle
import matplotlib.pyplot as plt
import numpy as np
